# 🤝 Ejercicio 3.2: Handoffs Dinámicos

**Módulo 3: Agentes Autónomos**  
**Tiempo estimado**: 12 minutos  
**Ejercicio**: 3.2

---

## 🎯 Objetivos de Aprendizaje

1. ✅ Crear tools de transferencia
2. ✅ Agentes que deciden cuándo transferir
3. ✅ Routing basado en tool_calls
4. ✅ Ciclos de handoff


In [ ]:
# ╔════════════════════════════════════════════════════════════╗
# ║  📋 SETUP Y VERIFICACIÓN                                  ║
# ╚════════════════════════════════════════════════════════════╝

%pip install -q langgraph langchain-openai python-dotenv

import os, sys
from dotenv import load_dotenv
sys.path.append(os.path.abspath('../..'))
load_dotenv()

print('='*50)
print('   SETUP VERIFICATION')
print('='*50)
print(f"✅ Python {sys.version.split()[0]}")
print(f"{'✅' if os.getenv('OPENAI_API_KEY') else '❌'} OpenAI API Key")
print(f"\n🎬 Ready!\n")

## 💬 SCRIPT

> "Handoffs: Los agentes se PASAN el control.
> Como un hospital: recepción → especialista → otro especialista."

In [ ]:
from typing import Annotated
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
import operator

class HandoffState(TypedDict):
    messages: Annotated[list, operator.add]
    next_agent: str

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
print('✅ Setup')

In [ ]:
# Tools de transferencia

@tool
def transfer_to_technical(reason: str) -> str:
    '''Transfiere al agente técnico. Usa cuando hay problemas técnicos.'''
    return f'TRANSFER:technical:{reason}'

@tool
def transfer_to_billing(reason: str) -> str:
    '''Transfiere al agente de billing. Usa para temas de facturación.'''
    return f'TRANSFER:billing:{reason}'

print('✅ Transfer tools definidas')

In [ ]:
# Agentes con handoff

support_tools = [transfer_to_technical, transfer_to_billing]
support_llm = llm.bind_tools(support_tools)

def support_agent(state):
    print('📞 Support agent...')
    response = support_llm.invoke(state['messages'])
    
    # ¿Decidió transferir?
    if response.tool_calls:
        call = response.tool_calls[0]
        if 'transfer_to' in call['name']:
            dest = call['name'].split('_')[-1]
            print(f'  → Transfiriendo a {dest}')
            return {'messages': [response], 'next_agent': dest}
    
    return {'messages': [response], 'next_agent': 'end'}

def technical_agent(state):
    print('🔧 Technical agent resolviendo...')
    response = llm.invoke(state['messages'] + [HumanMessage('(Agente técnico) Resuelvo el problema.')])
    return {'messages': [response], 'next_agent': 'support'}

def billing_agent(state):
    print('💰 Billing agent resolviendo...')
    response = llm.invoke(state['messages'] + [HumanMessage('(Agente billing) Resuelvo el problema.')])
    return {'messages': [response], 'next_agent': 'support'}

print('✅ Agentes con handoff definidos')

In [ ]:
# Routing de handoffs

def route_handoff(state):
    next_agent = state.get('next_agent', 'end')
    if next_agent == 'technical':
        return 'technical'
    elif next_agent == 'billing':
        return 'billing'
    elif next_agent == 'support':
        return 'support'
    return 'end'

print('✅ Router definido')

In [ ]:
# Construir grafo

graph = StateGraph(HandoffState)

graph.add_node('support', support_agent)
graph.add_node('technical', technical_agent)
graph.add_node('billing', billing_agent)

graph.add_edge(START, 'support')
graph.add_conditional_edges(
    'support',
    route_handoff,
    {
        'technical': 'technical',
        'billing': 'billing',
        'support': 'support',
        'end': END
    }
)
graph.add_edge('technical', 'support')
graph.add_edge('billing', 'support')

app = graph.compile()
print('✅ Sistema de handoffs compilado')

In [ ]:
# 🎬 DEMO

query = 'Mi API da error 500 y mi factura está incorrecta'

print('='*60)
print('🤝 HANDOFFS DINÁMICOS')
print('='*60)
print(f'Query: {query}\n')

result = app.invoke({'messages': [HumanMessage(query)], 'next_agent': ''})

print(f'\nMensajes intercambiados: {len(result["messages"])}')
print(f'Último mensaje: {result["messages"][-1].content[:100]}...')

---

## ✅ CHECKPOINT FINAL

- [ ] Código ejecuta sin errores
- [ ] Conceptos clave entendidos
- [ ] Listos para continuar

### 💬 PREGUNTA:
> "¿Alguna duda antes de continuar?"
